## Exercises

#### Data Acquisition

These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises use the cases, dept, and source tables from the 311_data on the Codeup MySQL server.

Read the case, department, and source data into their own spark dataframes.

Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

- Inspect your folder structure. What do you notice?

- Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

- How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

- How many Stray Animal cases are there?

- How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

- Convert the council_district column to a string column.

- Extract the year from the case_closed_date column.

- Convert num_days_late from days to hours in new columns num_hours_late.

- Join the case data with the source and department data.

- Are there any cases that do not have a request source?

- What are the top 10 service request types in terms of number of requests?

- What are the top 10 service request types in terms of average days late?

- Does number of days late depend on department?

- How do number of days late depend on department and request type?

- You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.

In [12]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import env
import os

In [3]:
def get_db_url(db, user=env.user, host=env.host, password=env.password):
    return (f'mysql+pymysql://{user}:{password}@{host}/{db}')

In [4]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/24 12:36:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
url = get_db_url('311_data')

In [6]:
query = 'select * from source'
source = pd.read_sql(query, url)
source = spark.createDataFrame(source)
source

DataFrame[index: bigint, source_id: string, source_username: string]

In [7]:
query = 'select * from cases'
df = pd.read_sql(query, url)
df = spark.createDataFrame(df)
df

DataFrame[case_id: bigint, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: bigint]

In [8]:
query = 'select * from dept'
dept = pd.read_sql(query, url)
dept = spark.createDataFrame(dept)
dept

DataFrame[dept_division: string, dept_name: string, standardized_dept_name: string, dept_subject_to_SLA: string]

In [9]:
source.write.json('data/sources_json', mode='overwrite')

In [10]:
source.write.csv('data/sources_csv', mode='overwrite')

In [13]:
os.listdir('data/sources_csv')

['.part-00002-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv.crc',
 '.part-00003-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv.crc',
 'part-00006-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv',
 '.part-00001-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv.crc',
 'part-00007-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv',
 '.part-00000-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv.crc',
 '._SUCCESS.crc',
 'part-00004-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv',
 'part-00005-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv',
 'part-00002-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv',
 'part-00003-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv',
 '.part-00007-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv.crc',
 '_SUCCESS',
 'part-00000-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv',
 '.part-00006-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv.crc',
 '.part-00004-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv.crc',
 'part-00001-24ba5fc4-2c09-45e9-b23d-6d95e234a81e-c000.csv',
 '.part-00005-24ba5

In [14]:
df.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [15]:
df = df.withColumnRenamed('SLA_due_date', 'case_due_date')

In [18]:
df = (
    df.withColumn('case_late', col('case_late') == 'YES')
    .withColumn('case_closed', col('case_closed') == 'YES')
)

df = df.withColumn('council_district', format_string('%03d', col('council_district')))

fmt = 'M/d/yy H:mm'
df = (
    df.withColumn('case_opened_date', to_timestamp(col('case_opened_date'), fmt))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), fmt))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), fmt))
)

In [25]:
df = df.withColumn('request_address', lower(trim(col('request_address'))))

In [26]:
df = df.withColumn('zipcode', regexp_extract(col('request_address'), r'\d+$', 0))

In [27]:
df = (
    df.withColumn('case_age', datediff(current_timestamp(), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed')
)

In [28]:
df.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'case_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district',
 'zipcode',
 'case_lifetime']

In [29]:
(
    df.select('case_closed', 'case_lifetime', 'num_days_late')
    .filter('! case_closed').sort(desc('num_days_late'))
    .where('num_days_late != "nan"')
).show(5)


22/10/24 13:26:17 WARN TaskSetManager: Stage 2 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-------------+------------------+
|case_closed|case_lifetime|     num_days_late|
+-----------+-------------+------------------+
|      false|         2122|       348.6458333|
|      false|         2122|       348.6458333|
|      false|         2121|348.52356480000003|
|      false|         2120|347.58256939999995|
|      false|         2118|       345.3894213|
+-----------+-------------+------------------+
only showing top 5 rows



In [30]:
(
    df.select('case_closed', 'case_lifetime', 'num_days_late')
    .filter('! case_closed').sort(desc('num_days_late'))
    .where('num_days_late != "nan"')
).show(5)

22/10/24 13:27:46 WARN TaskSetManager: Stage 3 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-------------+------------------+
|case_closed|case_lifetime|     num_days_late|
+-----------+-------------+------------------+
|      false|         2122|       348.6458333|
|      false|         2122|       348.6458333|
|      false|         2121|348.52356480000003|
|      false|         2120|347.58256939999995|
|      false|         2118|       345.3894213|
+-----------+-------------+------------------+
only showing top 5 rows



In [31]:
df.select('service_request_type').distinct().show()

22/10/24 13:28:16 WARN TaskSetManager: Stage 4 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+
|service_request_type|
+--------------------+
|Minimum Housing-O...|
|        Tree Removal|
| Service Information|
|    Sign Maintenance|
|Park Building Mai...|
|Brush Property Da...|
|Vacant Lot-Privat...|
|Graffiti: Private...|
|Traffic Sign Graf...|
|License Renewal I...|
|Guardrail- New Re...|
|Markings Installa...|
|   Sewer Line Broken|
|Zoning: Multi-Fam...|
|Engineering Inves...|
|    Zoning: Setbacks|
|  Traffic Sign Faded|
|     Permits, Fences|
|Certificates of O...|
|  Dead Animal - Misc|
+--------------------+
only showing top 20 rows



In [32]:
df.filter(expr('service_request_type == "Stray Animal"')).count()

22/10/24 13:28:48 WARN TaskSetManager: Stage 7 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


26760

In [33]:
(
    df.filter(df.dept_division == 'Field Operations')
    .filter(df.service_request_type != 'Officer Standby')
).count()

22/10/24 13:29:55 WARN TaskSetManager: Stage 10 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


113902

In [34]:
df.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('zipcode', 'string'),
 ('case_lifetime', 'int')]

In [35]:
df.select('case_closed_date').show(5)

22/10/24 13:30:25 WARN TaskSetManager: Stage 13 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


22/10/24 13:30:29 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 13 (TID 59): Attempting to kill Python Worker
+-------------------+
|   case_closed_date|
+-------------------+
|2018-01-01 12:29:00|
|2018-01-03 08:11:00|
|2018-01-02 07:57:00|
|2018-01-02 08:13:00|
|2018-01-01 13:29:00|
+-------------------+
only showing top 5 rows



In [36]:
df = df.withColumn('case_closed_year', year('case_closed_date'))

In [37]:
df.select('case_closed_date', 'case_closed_year').show(5)

22/10/24 13:31:29 WARN TaskSetManager: Stage 14 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


22/10/24 13:31:33 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 14 (TID 60): Attempting to kill Python Worker
+-------------------+----------------+
|   case_closed_date|case_closed_year|
+-------------------+----------------+
|2018-01-01 12:29:00|            2018|
|2018-01-03 08:11:00|            2018|
|2018-01-02 07:57:00|            2018|
|2018-01-02 08:13:00|            2018|
|2018-01-01 13:29:00|            2018|
+-------------------+----------------+
only showing top 5 rows



In [38]:
df.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'case_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district',
 'zipcode',
 'case_lifetime',
 'case_closed_year']

In [39]:
source.columns

['index', 'source_id', 'source_username']

In [40]:
dept.columns

['dept_division', 'dept_name', 'standardized_dept_name', 'dept_subject_to_SLA']

In [41]:
df = df.join(dept, 'dept_division', 'left').join(source, 'source_id', 'left')

In [42]:
df.columns

['source_id',
 'dept_division',
 'case_id',
 'case_opened_date',
 'case_closed_date',
 'case_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'service_request_type',
 'SLA_days',
 'case_status',
 'request_address',
 'council_district',
 'zipcode',
 'case_lifetime',
 'case_closed_year',
 'dept_name',
 'standardized_dept_name',
 'dept_subject_to_SLA',
 'index',
 'source_username']

In [43]:
df.filter(expr('index is null')).count()

22/10/24 13:33:13 WARN TaskSetManager: Stage 15 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


0

In [47]:
(df.groupby('service_request_type').count().sort(desc('count'))).show(10)

22/10/24 13:34:33 WARN TaskSetManager: Stage 39 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



In [48]:
(
    df.groupby('service_request_type')
    .agg(round(mean('num_days_late'), 2).alias('mean_days_late'))
    .sort(desc('mean_days_late'))
    .na.drop()
).show(10, truncate=False)

22/10/24 13:36:13 WARN TaskSetManager: Stage 51 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------------------+--------------+
|service_request_type                  |mean_days_late|
+--------------------------------------+--------------+
|Zoning: Junk Yards                    |175.96        |
|Labeling for Used Mattress            |162.43        |
|Record Keeping of Used Mattresses     |154.0         |
|Signage Requied for Sale of Used Mattr|151.64        |
|Storage of Used Mattress              |142.11        |
|Zoning: Recycle Yard                  |135.93        |
|Donation Container Enforcement        |131.76        |
|License Requied Used Mattress Sales   |128.8         |
|Traffic Signal Graffiti               |101.8         |
|Complaint                             |72.87         |
+--------------------------------------+--------------+
only showing top 10 rows



In [51]:
(
    df.groupby('service_request_type').avg().sort(desc('avg(num_days_late)')).na.drop()
    .select('service_request_type', col('avg(num_days_late)').alias('test')).show(10, truncate=False)
)

22/10/24 13:39:47 WARN TaskSetManager: Stage 63 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------------------+------------------+
|service_request_type                  |test              |
+--------------------------------------+------------------+
|Zoning: Junk Yards                    |175.95636210420952|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.9972403942857 |
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Storage of Used Mattress              |142.112556415     |
|Zoning: Recycle Yard                  |135.928516124798  |
|Donation Container Enforcement        |131.75610506358706|
|License Requied Used Mattress Sales   |128.79828704142858|
|Complaint                             |72.8705023031169  |
|Vendors                               |66.548098985078   |
+--------------------------------------+------------------+
only showing top 10 rows



In [53]:
df.groupby('dept_name').mean('num_days_late').show()

22/10/24 13:40:43 WARN TaskSetManager: Stage 75 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-------------------+
|           dept_name| avg(num_days_late)|
+--------------------+-------------------+
|Animal Care Services|-226.51783940550334|
|                null|   135.928516124798|
|Solid Waste Manag...|-2.2000575136721796|
|Development Services|  13.43372455586971|
|Trans & Cap Impro...|-20.612837354052626|
|    Customer Service|   59.7370914963008|
|        Metro Health| -4.911766979607001|
|Parks and Recreation| -5.251521960055137|
|Code Enforcement ...| -38.70133068329578|
|        City Council|                NaN|
+--------------------+-------------------+



In [54]:
df.select('dept_name').distinct().count()

22/10/24 13:41:05 WARN TaskSetManager: Stage 87 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


10

In [55]:
df.groupby('dept_name', 'service_request_type').mean('num_days_late').sort('service_request_type').show()

22/10/24 13:41:50 WARN TaskSetManager: Stage 100 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+--------------------+
|           dept_name|service_request_type|  avg(num_days_late)|
+--------------------+--------------------+--------------------+
|Solid Waste Manag...|1st Call Bagged L...|  -3.868991989858193|
|Solid Waste Manag...|2nd Call Bagged L...|   -4.21210716234353|
|Solid Waste Manag...|3rd Call Bagged L...|        -3.152896412|
|Trans & Cap Impro...|ADA Infrastructur...| -11.841508136999998|
|Code Enforcement ...|    ADA Obstructions|        -13.22349537|
|Code Enforcement ...|Absentee Property...| -14.231748829519532|
|Trans & Cap Impro...|Accident Problem ...| -22.971030459931214|
|Solid Waste Manag...|Additional Cart R...|  -5.063509766847671|
|Solid Waste Manag...|Additional Garbag...|  -2.920198210360159|
|Solid Waste Manag...|Additional Organi...|  -5.997874052909091|
|Animal Care Services|Aggressive Animal...|  16.696368811892242|
|Animal Care Services|Aggressive Animal...|  2.6450338831632556|
|Trans & Cap Impro...|  A